In [1]:
# Code to be able to import local modules in notebooks
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:

# Import dependencies
# Standard python libraries
import os
import json

from oauthlib.oauth2 import WebApplicationClient
import requests
import pandas as pd
import plotly

# Internal imports
from ffpackage.scraping import mfl
from ffpackage.analysis import analysis
# from ffpackage.scraping import ffdb
# from ffpackage.scraping import ourlads
from ffpackage.viz import viz


from appmanager.database import db
from appmanager.user import user




# Configuration (These variables are stored as environment variables)
GOOGLE_CLIENT_ID = os.environ.get("GOOGLE_CLIENT_ID", None)
GOOGLE_CLIENT_SECRET = os.environ.get("GOOGLE_CLIENT_SECRET", None)
GOOGLE_DISCOVERY_URL = ("https://accounts.google.com/.well-known/openid-configuration")

In [3]:
#def getFranchise():
user_league = "53906"

franchises = mfl.get_franchises(user_league)['franchiseName'].to_list()

franchises

['Providence Monastic Spirits',
 'Washington Fantasy Football Team',
 'Verdansk Vaqueros FFC',
 'Down with the Sickness unto Death',
 'Crocodilopolis Body Snatchers',
 "O'Henry's Stiffies",
 'Comeback Kings',
 'WHAM! BAM! Thank You CAM!',
 'Pretty Big Wieners',
 'Saskatoon Squatches',
 'Gus The Bus',
 'IDK much about soccer ']

In [4]:
#def allPlayers():
player_df = db.get_df("player_df")
tables = [player_df.to_html(classes='data')]
titles=player_df.columns.values
#return render_template("allPlayers.html", tables=tables, titles=titles)

/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [5]:
#@app.route('/waiverWire', methods=['GET', 'POST'])
#@login_required
#def waiverWire():
user_league = "53906"
user_franchise = "0005"

# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
# Append a row to carry free agents
franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent"}, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league, user_franchise=user_franchise)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Combine Franchise rosters with free agents to get all players
rosters_df = pd.concat([rosters_df, freeAgent_df], axis=0)

# Get all players, sharkRank, and ADP
player_df = db.get_df("predictions")

# Merge all dfs
complete = player_df.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName']], on='franchiseID', how='left')
complete = complete[complete['franchiseID'].notna()]
complete = complete.sort_values(by=['pred'], ascending=False)
complete.reset_index(inplace=True, drop=True)
complete = complete[['player', 'age', 'team', 'franchiseName', 'pos', 'posRank', 'KR', 'PR', 'RES', 'pred', 'sharkAbsolute', 'adpAbsolute']]
complete.rename(columns={
    'player':'Player',
    'age':'Age',
    'team':'Team',
    'pos':'Position',
    'posRank': 'Rank',
    'pred': 'My Prediction',
    'sharkAbsolute': 'FantasySharks Prediction',
    'adpAbsolute': 'ADP-Based Prediction'
}, inplace=True)
complete.set_index('Player', drop=True, inplace=True)

#return render_template("waiverWire.html", tables=[complete.to_html(classes='data')], titles=complete.columns.values)

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_26407/2224525526.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent"}, ignore_index=True)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [3]:
#@app.route('/compareFranchises')
#@login_required
#def compareFranchises():
user_league = "53906"

# Get Franchises in the league
franchise_df = mfl.get_franchises(user_league)
# Append a franchise for free agents
franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent", "franchiseAbbrev":"FA"}, ignore_index=True)

# Get franchise rosters
rosters_df = mfl.get_rosters(user_league)

# Get Free Agents
freeAgent_df = mfl.get_freeAgents(user_league)

# Get all players, sharkRank, and ADP
predictions = db.get_df("predictions")

# Merge all dfs
complete = predictions.merge(rosters_df, on='id_mfl', how='left').merge(franchise_df[['franchiseID', 'franchiseName', 'franchiseAbbrev']], on='franchiseID', how='left')
complete['franchiseID'].fillna("FA", inplace=True)
complete['franchiseName'].fillna("Free Agent", inplace=True)
complete['rosterStatus'].fillna("Free Agent", inplace=True)

# Get info on available slots from MFL site
posMax = {"QB":2, "RB":5, "WR":6, "TE":5, "PK":2, "DF":2}
posMin = {"QB":1, "RB":2, "WR":2, "TE":2, "PK":2, "DF":2}
totalStarters = 15
predMethod = "pred"

# Select starters
df = analysis.starterSelector(complete, how=predMethod, startersMax=totalStarters, posMax=posMax, posMin=posMin)

# Find the lowest scoring player on the field and set them as the low bar
for x in ["QB", "RB", "WR", "TE", "PK", "DF"]:
    positionMinPred = df.loc[(df['pos']==x) & (df['starting']=='Starter'), predMethod].min()
    # Calculate relative values
    df.loc[df['pos']==x, 'relativeValue'] = df.loc[df['pos']==x, predMethod] - positionMinPred

# Visualize the data
fig = viz.compareFranchises(df, how=predMethod)
# Encode in JSON format
graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
# Render html template in flask
#return render_template('compareFranchises.html', graphJSON=graphJSON)

/var/folders/ss/nt9sccdn5w91sbtr85gybrnm0000gn/T/ipykernel_26731/450361586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  franchise_df = franchise_df.append({"franchiseID":"FA", "franchiseName":"Free Agent", "franchiseAbbrev":"FA"}, ignore_index=True)
/Users/travisharry/Desktop/Coding/ff_demo2/appmanager/database/db.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query, conn)


In [ ]:
#@app.route('/liveScoring')
#@login_required
#def liveScoring():
user_league = "53906"

# Get MFL scoring data
liveScores = get_mfl_liveScoring(user_league)
# Get Franchises in the league
franchises = get_mfl_league(user_league)
# Get all players, sharkRank, and ADP
predictions = get_df("predictions")

# merge predictions, franchises, and liveScores
merged = liveScores.merge(franchises, how='left', on='franchiseID').merge(predictions, how='left', on='id_mfl')
# Clean
merged['liveScore'] = merged.liveScore.astype('float64')
merged['secondsRemaining'] = merged.secondsRemaining.astype('float64')

# calculate scoreRemaining
merged['weeklyPred'] = merged['pred'] / 17
def calcScoreRemaining(row):
    result = ((row['weeklyPred']) * (row['secondsRemaining'] / 3600)) + row['liveScore']
    return result
merged['scoreRemaining'] = merged.apply(calcScoreRemaining, axis=1)
# Claculate difference between projection/actual
merged['diff'] = merged.scoreRemaining - merged.weeklyPred
# Normalize difference
merged.loc[merged['diff']>20, 'diff'] = 20
merged.loc[merged['diff']<-20, 'diff'] = -20
merged['scaled'] = round(merged['diff'] * 255 / 20, 0)
merged.dropna(inplace=True)
merged['scaled'] = merged['scaled'].astype('int')
# Set colors for chart
def colorPicker(row):
    scalar = row['scaled']
    if scalar >= 0:
        red = 255 - scalar
        green = 255
        blue = 255 - scalar
    else:
        red = 255 
        green = 255 + scalar
        blue = 255 + scalar
    color = f'rgb({red},{green},{blue})'
    return color
merged['color'] = merged.apply(colorPicker, axis=1)
# chart
players_onthefield = merged.loc[merged.status=="starter"]
players_onthefield = players_onthefield.sort_values(by='scoreRemaining', ascending=False, ignore_index=True)
fran_rank = players_onthefield.groupby('franchiseName').sum().sort_values(by='scoreRemaining', ascending=False)
sorter = fran_rank.index
players_onthefield.franchiseName = players_onthefield.franchiseName.astype("category")
players_onthefield.franchiseName.cat.set_categories(sorter, inplace=True)
players_onthefield.sort_values(["franchiseName"], inplace=True)
color_discrete_map = dict(zip(players_onthefield.id_mfl, players_onthefield.color))
# Create bar chart
figLive = px.bar(players_onthefield, 
            x="franchiseName", 
            y="scoreRemaining", 
            color="player", 
            color_discrete_sequence=list(players_onthefield['color']),
            category_orders={
                "pos": ["QB", "RB", "WR", "TE", "PK", "DF"]},
            text='player', 
            hover_name="player",
            hover_data={
                'scoreRemaining':True, 'weeklyPred':True, 'scaled':True,
                'player':False, 'pos':False, 'franchiseName':False
                },
            labels={
                "franchiseName":"Franchise",
                "predRelative":"Player Value",
                "pred":"ChopBlock Prediction",
                "sharkAbsolute":"FantasySharks Prediction",
                "adpAbsolute":"ADP-Based Prediction"
            }
            )
figLive.update_layout(
            barmode='stack', 
            xaxis={'categoryorder':'total descending'},
            plot_bgcolor='rgba(0,0,0,0)',
            title="ChopBlock Predictions",
            font_family="Skia",
            showlegend=False
            )
# Encode in JSON format
graphJSON = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
# Render html template in flask
#return render_template('liveScoring.html', graphJSON=graphJSON)